In [5]:
import pandas as pd
import numpy as np
import json
import ast

In [9]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

## Merge both datasets by title

In [10]:
movies = movies.merge(credits,on='title')
movies = movies[['genres','id','keywords','title','overview','cast','crew']]

In [12]:
movies.dropna(inplace=True)


In [13]:
def tagger(a):
    l = []
    for i in ast.literal_eval(a):
        l.append(i['name'])
    return l

def tagger3(a):
    l = []
    count = 0
    for i in ast.literal_eval(a):
        if count!=3:
            l.append(i['name'])
            count+=1
        else:
            break
    return l
    
def tagger_direct(a):
    l = []
    for i in ast.literal_eval(a):
        if i['job']=='Director':
            l.append(i['name'])
    return l


In [14]:
movies['genres'] = movies['genres'].apply(tagger)
movies['keywords'] = movies['keywords'].apply(tagger)
movies['cast']=movies['cast'].apply(tagger3)
movies['crew'] = movies['crew'].apply(tagger_direct)

In [16]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [17]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [18]:
movies['tag'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [23]:
m2 = movies[['id','title','tag']]
m2['tag'] = m2['tag'].apply(lambda x:" ".join(x))
m2['tag'] = m2['tag'].apply(lambda x:x.lower())

C:\Users\hp\AppData\Local\Temp\ipykernel_6356\391863087.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m2['tag'] = m2['tag'].apply(lambda x:" ".join(x))
C:\Users\hp\AppData\Local\Temp\ipykernel_6356\391863087.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m2['tag'] = m2['tag'].apply(lambda x:x.lower())


In [24]:
m2['tag'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

## STEMMING

In [25]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [26]:
def stemmer(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [27]:
m2['tag'] = m2['tag'].apply(stemmer)

C:\Users\hp\AppData\Local\Temp\ipykernel_6356\357072221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m2['tag'] = m2['tag'].apply(stemmer)


## Count Vectorizer

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000,stop_words = 'english')

In [31]:
vect = cv.fit_transform(m2['tag']).toarray()

## Cosine similarity

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
sim = cosine_similarity(vect)

In [35]:
def recommender(movie):
    ind = m2[m2['title']==movie].index[0]
    dist = list(enumerate(sim[ind]))
    rev = sorted(dist,reverse=True,key = lambda x:x[1])
    for i in rev[1:6]:
        print(i,"\t",m2.iloc[i[0]].title)

In [36]:
recommender('Batman Begins')

(65, 0.40218090755486674) 	 The Dark Knight
(1363, 0.35434169344615046) 	 Batman
(1362, 0.3340765523905305) 	 Batman
(3, 0.3177444546511212) 	 The Dark Knight Rises
(3297, 0.3120099844792576) 	 10th & Wolf


In [37]:
import pickle

In [38]:
#copy these files into virtual environment
pickle.dump(m2.to_dict(),open('movies_dict.pkl','wb'))
pickle.dump(sim,open('similarities.pkl','wb'))